# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collections = db.list_collection_names()

In [6]:
# review the collections in our new database
for collection in collections:
    print(collection)

establishments


In [7]:
# review a document in the establishments collection
document = db['establishments'].find_one()
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a98c6174f143cb5253ddf5'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [8]:
# assign the collection to a variable
establishments = db['establishments']

In [9]:
# # Drop all documents in the 'establishments' collection
# db['establishments'].drop()


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Delete all existing instances of 'Penang Flavours' (for re-running code purposes)
delete_result = establishments.delete_many({'BusinessName': 'Penang Flavours'})

if delete_result.deleted_count > 0:
    print("All existing instances of 'Penang Flavours' deleted successfully.")
else:
    print("No existing instances of 'Penang Flavours' found.")

No existing instances of 'Penang Flavours' found.


In [11]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [12]:
# Insert the new restaurant into the collection
result = establishments.insert_one(penang_flavours)

In [13]:
# Check that the new restaurant was inserted

penang_flavours_document = establishments.find_one({'BusinessName': 'Penang Flavours'})

# Print the document if found
if penang_flavours_document:
    pprint(penang_flavours_document)
else:
    print("Penang Flavours not found.")


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a98cb5c4110413d90bccb0'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [14]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessName': 'Penang Flavours'}
projection = {'BusinessTypeID': 0, 'BusinessType': 1, '_id': 0}

# Find and print the document with the specified fields
penang_flavours_info = establishments.find_one(query, {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0})

if penang_flavours_info:
    pprint(penang_flavours_info)
else:
    print("Penang Flavours not found.")

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [15]:
# Update the new restaurant with the correct BusinessTypeID

# Define the filter for finding the document to update
filter_criteria = {'BusinessName': 'Penang Flavours'}

# Define the update operation to set the correct BusinessTypeID
update_operation = {'$set': {'BusinessTypeID': 1}}

# Update the document in the 'establishments' collection
update_result = establishments.update_one(filter_criteria, update_operation)

In [16]:
# Confirm that the new restaurant was updated

if update_result.modified_count > 0:
    print("Restaurant 'Penang Flavours' updated successfully.")
else:
    print("Restaurant 'Penang Flavours' not found or no update needed.")

# Query the 'establishments' collection to find the specified restaurant
penang_flavours_info = establishments.find_one({'BusinessName': 'Penang Flavours'}, {'BusinessTypeID': 1, '_id': 0})

# Print the document if found
if penang_flavours_info:
    pprint(penang_flavours_info)
else:
    print("Penang Flavours not found in the 'establishments' collection.")


Restaurant 'Penang Flavours' updated successfully.
{'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"

# Count the number of documents with LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})

# Print the count
print("Number of documents with LocalAuthorityName 'Dover':", dover_count)


Number of documents with LocalAuthorityName 'Dover': 994


In [18]:
# Delete all documents where LocalAuthorityName is "Dover"

# Define the filter for deleting documents
filter_criteria = {'LocalAuthorityName': 'Dover'}

# Delete all documents with LocalAuthorityName as "Dover"
delete_result = establishments.delete_many(filter_criteria)

# Print the number of deleted documents
print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 994


In [19]:
# Check if any remaining documents include Dover

# Count the number of documents with LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})

# Print the count
print("Number of documents with LocalAuthorityName 'Dover':", dover_count)

Number of documents with LocalAuthorityName 'Dover': 0


In [20]:
# Check that other documents remain with 'find_one'

# Find and print a single document after the deletion
remaining_document = establishments.find_one()

# Print the document if found
if remaining_document:
    pprint(remaining_document)
else:
    print("No remaining documents in the 'establishments' collection.")


{'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289352,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2015-08-27T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a98c6174f143cb5253e0dc'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289352',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [21]:
# Change the data type from String to Decimal for longitude and latitude

# Find the number of documents with geocode in string format
filter_criteria = {'$or': [
    {'geocode.latitude': {'$type': 'string'}},
    {'geocode.longitude': {'$type': 'string'}}
]}

# Count the number of documents matching the filter criteria
num_documents_with_strings = establishments.count_documents(filter_criteria)

# Print the result
print("Number of documents with 'geocode' in string format:", num_documents_with_strings)


Number of documents with 'geocode' in string format: 38786


In [22]:
from bson.decimal128 import Decimal128

# Change the data type from String to Decimal for longitude and latitude

# Define the filter criteria to select documents with string latitude or longitude
filter_criteria = {'$or': [
    {'geocode.latitude': {'$type': 'string'}},
    {'geocode.longitude': {'$type': 'string'}}
]}

# Retrieve the documents that need updating
documents_to_update = establishments.find(filter_criteria)

# Iterate through the documents and update the 'geocode' field
for document in documents_to_update:
    # Check if latitude is a string and convert if necessary
    if isinstance(document['geocode']['latitude'], str):
        document['geocode']['latitude'] = Decimal128(document['geocode']['latitude'])

    # Check if longitude is a string and convert if necessary
    if isinstance(document['geocode']['longitude'], str):
        document['geocode']['longitude'] = Decimal128(document['geocode']['longitude'])

    # Update the document in the 'establishments' collection
    establishments.update_one({'_id': document['_id']}, {'$set': {'geocode': document['geocode']}})

# Print a message after the update
print("Conversion to decimals completed.")


Conversion to decimals completed.


In [23]:
# Checks if 'geocode' was updated to a decimal
document = db['establishments'].find_one()
pprint(document)

{'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289352,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2015-08-27T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65a98c6174f143cb5253e0dc'),
 'geocode': {'latitude': Decimal128('51.08084'),
             'longitude': Decimal128('1.188537')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289352',
            'rel': 'self'}],
 'meta': {'dataSo

In [24]:
# Testing the extraction of latitude and longitude
updated_document = establishments.find_one()

if updated_document and 'geocode' in updated_document:
    # Access the 'geocode' field
    geocode = updated_document['geocode']

    # Extract 'latitude' and 'longitude' values as decimals
    latitude_decimal = geocode['latitude'].to_decimal()
    longitude_decimal = geocode['longitude'].to_decimal()

    # Print the decimal values
    print("Latitude as Decimal:", latitude_decimal)
    print("Longitude as Decimal:", longitude_decimal)
else:
    print("No documents found with 'geocode' field.")


Latitude as Decimal: 51.08084
Longitude as Decimal: 1.188537


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [26]:
# Change the data type from String to Integer for RatingValue

# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, {'$set': {"RatingValue": None}})

# Convert RatingValue to Integer for valid numeric strings
establishments.update_many({
    "RatingValue": {
        "$exists": True,
        "$type": "string",  # Check if it's a string
        "$regex": "^[1-5]$"  # Check if it's a valid numeric string (1-5)
    }
}, [{'$set': {"RatingValue": {'$toInt': "$RatingValue"}}}])



UpdateResult({'n': 34629, 'nModified': 34629, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [27]:
# Check that the coordinates and rating value are now numbers

sample_document = establishments.find_one()

# Extract geocode values
geocode = sample_document.get('geocode', {})
latitude_value = geocode.get('latitude', Decimal128('0')).to_decimal()
longitude_value = geocode.get('longitude', Decimal128('0')).to_decimal()

# Extract RatingValue
rating_value_value = sample_document.get('RatingValue', None)

# Check if RatingValue is an integer or a valid numeric string
if isinstance(rating_value_value, int):
    rating_value_type = type(rating_value_value)
else:
    rating_value_value = int(rating_value_value) if rating_value_value.isdigit() else None
    rating_value_type = type(rating_value_value) if rating_value_value is not None else None

# Print the results
print("Latitude value:", latitude_value)
print("Longitude value:", longitude_value)
print("RatingValue value:", rating_value_value)

# Check data types
latitude_type = type(latitude_value)
longitude_type = type(longitude_value)

# Print the data types
print("Latitude data type:", latitude_type)
print("Longitude data type:", longitude_type)
print("RatingValue data type:", rating_value_type)


Latitude value: 51.08084
Longitude value: 1.188537
RatingValue value: 5
Latitude data type: <class 'decimal.Decimal'>
Longitude data type: <class 'decimal.Decimal'>
RatingValue data type: <class 'int'>
